### Step 1
API KEY: 0i-_-6pigerNnnRaU8_oiybRZz_UxMQuBHpE_copxSdw
Project Id: b596c884 -f867 -4771 -afcc-f9fd10dae1a4

API キーとプロジェクト ID を使用して、watsonx 経由でモデルにアクセスします。
```
import os
from getpass import getpass
watsonx_api_key = getpass()
os.environ["WATSONX_APIKEY"] = watsonx_api_key
```
これにより、watsonx キーの入力を求められます。
```
watsonx_project_id = getpass()
os.environ["WATSONX_PROJECT_ID"] = watsonx_project_id
```

### step2. 
WatsonxLLM は、LlamaIndex が提供する watsonx サービスへのインターフェースとして構成できます。WatsonxLLM オブジェクトは、使用するモデルと、そのモデルが使用すべきプロジェクトを設定します。ここでは、Granite 3 8-billion parameter Instruct モデルを使用します。

In [ ]:
from llama_index.llms.ibm import WatsonxLLM
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames
rag_gen_parameters = {
GenTextParamsMetaNames.DECODING_METHOD: "sample",
GenTextParamsMetaNames.MIN_NEW_TOKENS: 150,
GenTextParamsMetaNames.TEMPERATURE: 0.5,
GenTextParamsMetaNames.TOP_K: 5,
GenTextParamsMetaNames.TOP_P: 0.7
}
watsonx_llm = WatsonxLLM(
model_id="ibm/granite-3 -8b -instruct",
url="https://us-south.ml.cloud.ibm.com",
project_id=os.getenv("WATSONX_PROJECT_ID"),
max_new_tokens=512,
params=rag_gen_parameters,
)

パラメータは、モデルの出力をどのように構成するかを決定します。
具体的には以下のような意味を持ちます：
• DECODING_METHOD
応答を生成する際の単語選択方法を指定します。
"sample"にすると確率分布に基づきランダム性を持たせた選択を行い、多様
な文章を生成します。
"greedy"の場合は常に最も確率が高い単語を選び、安定した出力になりま
す。
• MIN_NEW_TOKENS
モデルが新たに生成するトークン（単語や記号）の最小数を指定します。
ここでは 150 に設定し、応答が短くなりすぎないようにします。
• LLM temperature
出力のランダム性を制御するパラメータで、0〜1 の範囲を取ります。
低くすると（例：0.5）より予測可能で安定した文章になり、高くすると多
様性が増します。
• top_k
各生成ステップで候補とする単語数の上限を指定します。
値を小さく（例：5）すると、高確率の単語のみが選ばれ、文章が安定しや
すくなります。
• top_p
累積確率が指定値（例：0.7）以下となる候補単語のみを選択します。
これにより、意味的な一貫性を保ちながらも、適度な変動を持つ出力が得
られます。
API キーを正しく設定すれば、エラーは発生しないはずです

### Step 3
Jupyter ノートブックの中で動く「イベントループ」と、LlamaIndex の RAG 処理で使う「イベントループ」は別々に動いています。
このままだと、うまく動かないことがあります。
そこで、Python の asyncio というライブラリを使って、自分だけの新しいイベントループを作り、両方が問題なく動くようにします。

In [ ]:
import asyncio, nest_asyncio
nest_asyncio.apply()
loop = asyncio.get_event_loop()

IBM の年次報告書をダウンロードして保存し、PyMuPDFReader のインスタンスに読み込ませます。これで文書を解析し、ベクトルストアに取り込むための埋め込み（embeddings）を作成できます。

プロジェクトのルートに「docs」フォルダを作成してください（作成後プロジェクトのルート: プロジェクト名/docs） 。以下のスクリプトは、そのフォルダに PDFファイルをダウンロードします

In [ ]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader
import requests
def load_pdf(url):
r = requests.get(url)
name = url.rsplit('/', 1)[1]
# save to a docs dir
with open('docs/' + name, 'wb') as f:
f.write(r.content)
loader = PyMuPDFReader()
return loader.load(file_path="./docs/" + name)
pdf_doc =
load_pdf("https://www.ibm.com/annualreport/assets/downloads/IBM_Annual_Re
port_2023.pdf")